# Shapenet Rendering
- Blender script source: https://github.com/panmari/stanford-shapenet-renderer

## Semantic Segmentation
If rendering a single object, we get a mask (interpretted as a binary segmentation map) for free. However, if we want to render multiple objects, either for rendering our target object into a 3D scene or for using secondary models to produce oclusions, then we need to do something more. 

Here are some references for how we can do semantic segmentation:
- https://blender.stackexchange.com/questions/79595/change-diffuse-shader-to-emission-shader-without-affecting-shader-color
- https://blender.stackexchange.com/questions/80906/create-a-segmentation-picture-with-each-object-class-rendered-in-different-color/162746#162746
- https://blender.stackexchange.com/questions/34609/is-there-a-way-to-streamline-scripting-these-shaders-and-modifier-keyframes
- https://github.com/DIYer22/bpycv#install
 - This seems promising, but I had issues when trying to install openexr

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
import glob
import cv2
import imageio
import shutil

import subprocess
import numpy as np

from modules.ShapeNetHandler import ShapeNetHandler
import modules.utils

from IPython.display import Image, display

# Load ShapeNet Handler

In [3]:
shapenet_root = "/hdd/mliuzzolino/ShapeNet/data/ShapeNetCore.v2"
shapenet_handler = ShapeNetHandler(shapenet_root)

## Print available categories by name
- Can use name to condition random sampling

In [4]:
# shapenet_handler.print_categories()

## Randomly Sample a filepath

In [5]:
temp_obj_filepath, temp_synset_id, temp_instance_id, temp_obj_name = shapenet_handler.sample_obj(category_name='car')

## Output Path

In [6]:
_OUTPUT_ROOT = 'render_output'

# Render

In [7]:
_N_VIEW_ANGLES = 72
_N_MODELS_TO_RENDER = 18
_RANDOM_SCALING = False
_OVERWRITE = True
_RENDER_ENGINE = 'BLENDER_RENDER' # 'CYCLES', 'BLENDER_RENDER'
_RENDER_SCRIPT = "scripts/blender_render.py"
_TARGET_CATEGORY_LIST = ['airplane', 'car']

## Run Blender

In [ ]:
subprocess_outputs = []
try:
    for img_i in range(_N_MODELS_TO_RENDER):
        # Randomly select target category
        target_category = ''
        if _TARGET_CATEGORY_LIST != []:
            target_category = np.random.choice(_TARGET_CATEGORY_LIST)
        
        # Sample object filepath
        obj_filepath, synset_id, instance_id, obj_name = shapenet_handler.sample_obj(category_name=target_category)
        
        # Setup outpath
        output_root = os.path.join(_OUTPUT_ROOT, _RENDER_ENGINE, f'synsetID_{synset_id}', instance_id)
        
        # Randomly select obj scaling
        random_scale = np.random.uniform(0.99, 1.15) if _RANDOM_SCALING else 1.0
        random_n_occlusions = np.random.randint(3, 7)
        
        if not os.path.exists(output_root) or _OVERWRITE:
            # Remove previous directory if overwrite
            if _OVERWRITE and os.path.exists(output_root):
                shutil.rmtree(output_root)
            
            # Run blender script
            cmd = f"blender --background --python {_RENDER_SCRIPT} -- "
            cmd += f"--output_folder {output_root} --views {_N_VIEW_ANGLES} "
            cmd += f"--scale {random_scale} --n_occlusions {random_n_occlusions} "
            cmd += f"--render_engine {_RENDER_ENGINE} "
            cmd += f"{obj_filepath}"
            out = subprocess.check_output(cmd.split(" "), shell=False)
            subprocess_outputs.append(out)
        else:
            print("Skipping.")
        
        # Stdout
        stdout_str = f"[{img_i+1}/{_N_MODELS_TO_RENDER}] -- "
        stdout_str += f"SynsetID: {synset_id} -- "
        stdout_str += f"InstanceID: {instance_id:32s} -- "
        stdout_str += f"Name: {obj_name}"
        print(stdout_str)
        
except KeyboardInterrupt:
    try:
        # Try to cleanup directory with incomplete # renders
        if len(os.listdir(output_root)) != _N_VIEW_ANGLES:
            shutil.rmtree(output_root)
    except:
        pass
    print("\nEnding early.")

[1/18] -- SynsetID: 02958343 -- InstanceID: 744f3661d9030c059cc32a851e7d6f32 -- Name: car,auto,automobile,machine,motorcar


## Background Images

In [ ]:
background_imgs_root = '/hdd/mliuzzolino/Places2/places365_standard/train'
background_img_paths = glob.glob(f'{background_imgs_root}/*/*')

## Load Images and Animate

In [ ]:
_LOAD_LIMIT = 18

limit_reached = False

all_imgs = []
rendered_instance_ids = glob.glob(f"{os.path.join(_OUTPUT_ROOT, _RENDER_ENGINE)}/*/*")
np.random.shuffle(rendered_instance_ids)
for rendered_instance_id in rendered_instance_ids:
    sys.stdout.write(f"\rProcessed model {len(all_imgs)+1}/{_LOAD_LIMIT}...")
    sys.stdout.flush()

    # Load paths
    rendered_img_paths = np.sort(glob.glob(f"{rendered_instance_id}/*RGBA_full.png"))
    semantic_seg_paths = [ele.replace('_full', '_semantic_segmentation') for ele in rendered_img_paths]

    # Load images
    rendered_imgs = [modules.utils.read_image(img_path) 
                         for img_path in rendered_img_paths]
    segmentation_imgs = [modules.utils.read_image(img_path)
                         for img_path in semantic_seg_paths]

    # Ensure correct number of view angles present
    if len(rendered_imgs) != _N_VIEW_ANGLES:
        print(f"\t**Error: Insufficient angles! Actual: {len(rendered_imgs)} - Expected: {_N_VIEW_ANGLES}")
        continue

    # Setup background image
    background_img = modules.utils.sample_background_img(background_img_paths, shape=rendered_imgs[0].shape)

    # Apply background image
    rendered_imgs = [modules.utils.add_background(rgb_img, seg_img, background_img) 
                         for rgb_img, seg_img in zip(rendered_imgs, segmentation_imgs)]

    # Join rgb image with semantic seg image
    rendered_imgs = [(rgb, mask) for rgb, mask in zip(rendered_imgs, segmentation_imgs)]

    # Combine images
    combined_imgs = modules.utils.combine_imgs(rendered_imgs)
    all_imgs.append(combined_imgs)

    # Check limit
    if len(all_imgs) >= _LOAD_LIMIT:
        print("\nLimit reached. Ending early.")
        limit_reached = True
        break

In [ ]:
# Stack all images
stacked_imgs = modules.utils.stack_all_imgs(all_imgs, nrow=3)

In [ ]:
savepath = 'test.gif'
animation = modules.utils.toAnimation(stacked_imgs, figsize=(12,12), interval=100, savepath=savepath, fps=15)

In [ ]:
if savepath.endswith('gif'):
    display(Image(filename=savepath))